<a href="https://colab.research.google.com/github/kkattt51/ArtInt/blob/main/1_%22simple_chatbot_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install pymorphy2
!pip install pymorphy2-dicts

     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 8.2MB 13.3MB/s 
     |████████████████████████████████| 7.1MB 5.8MB/s 


In [ ]:
import nltk
import numpy as np
import random
import string

In [ ]:
with open('article.txt','r',errors = 'ignore') as f:
  raw=f.read()

FileNotFoundError: ignored

In [ ]:
raw = raw.lower() 

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [ ]:
sent_tokens[:300]

['цель машинного обучения — предсказать результат по входным данным.',
 'чем разнообразнее входные данные, тем проще машине найти закономерности и тем точнее результат.',
 'итак, если мы хотим обучить машину, нам нужны три вещи:\n\nданные хотим определять спам — нужны примеры спам-писем, предсказывать курс акций — нужна история цен, узнать интересы пользователя — нужны его лайки или посты.',
 'данных нужно как можно больше.',
 'десятки тысяч примеров — это самый злой минимум для отчаянных.',
 'данные собирают как могут.',
 'кто-то вручную — получается дольше, меньше, зато без ошибок.',
 'кто-то автоматически — просто сливает машине всё, что нашлось, и верит в лучшее.',
 'самые хитрые, типа гугла, используют своих же пользователей для бесплатной разметки.',
 'вспомните recaptcha, которая иногда требует «найти на фотографии все дорожные знаки» — это оно и есть.',
 'за хорошими наборами данных (датасетами) идёт большая охота.',
 'крупные компании, бывает, раскрывают свои алгоритмы, но дат

In [ ]:
word_tokens[:5]

['цель', 'машинного', 'обучения', '—', 'предсказать']

In [ ]:
import pymorphy2

lemmer = pymorphy2.MorphAnalyzer()

In [ ]:
# from nltk.stem.wordnet import WordNetLemmatizer

# eng_lemmer = WordNetLemmatizer()

In [ ]:
# eng_lemmer.lemmatize('going')

'going'

In [ ]:
def LemTokens(tokens):
    return [lemmer.parse(token)[0].normal_form for token in tokens]
    # return [eng_lemmer.lemmatize(token) for token in tokens]
    
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
import re 

# Тут нужно менять!
GREETING_INPUTS = ("привет", "здравствуйте", "здорово", "чо каво", "хай", "салют", "хаюшки", "сап")
GREETING_RESPONSES = ["привет", "хай", "*подмигнул*", "здравствуй", "рад общению с тобой", "приветствую"]

def greeting(sentence):
    sentence = re.sub('[^А-Яа-яё ]', '', sentence)
    #sentence = re.sub('[^A-Za-z ]', '', sentence)
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [ ]:
def response(user_response):
    robo_response=''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words("russian"))
    # TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Извините, но я не понимаю вас!" # ТУТ можно менять :) 
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [ ]:
import warnings
warnings.filterwarnings('ignore')

flag=True

# Тут тоже нужно менять!
print("Робот: Меня зовут Зак. Я буду отвечать на ваши вопросы о Машинном обучении. Если вы захотите прекратить общение, напишите Пока!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='пока'):
        if(user_response in ['спасибо', 'благодарю']): # И тут меняйте :)
            flag=False
            print("Робот: Рад был помочь ;)") # Меняйте :) 
        else:
            if(greeting(user_response)!=None):
                print("Робот: "+greeting(user_response))
            else:
                sent_tokens.append(user_response)
                word_tokens=word_tokens+nltk.word_tokenize(user_response)
                final_words=list(set(word_tokens))
                print("Робот: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Робот: До свидания! *машет-ручкой*") # Тут тоже нужно импровизировать

Робот: Меня зовут Зак. Я буду отвечать на твои вопросы о Чат-ботах. Если ты захочешь прекратить общение, напиши Пока!
здорово
Робот: рад общению с тобой
виды машинного обучения
Робот: важный, но не единственный.нейросети — один из видов машинного обучения.
обучение с учителем
Робот: либо, можно попробовать обучение без учителя.
что ты знаешь о рекуррентных нейросетях
Робот: про google не знаю.
рекуррентные нейросети
Робот: дай нейросети удочку — она сможет определять и удочку до конца жизни...

рекуррентные нейросети (rnn)
вторая по популярности архитектура на сегодняшний день.
популярные архитектуры
Робот: популярный, но есть и другие, не хуже.глубокое обучение — архитектура нейросетей, один из подходов к их построению и обучению.
нейросети и глубокое обучение
Робот: отличие глубокого обучения от классических нейросетей было в новых методах обучения, которые справлялись с большими размерами сетей.
что такое бустинг
Робот: вот это как раз оно, яндекс сейчас весь на бустинге.
пример бег

In [ ]:
Rasa

sklearn
tensorflow
torch
transformers
mystem, pymorphy2, spacy
gensim

NameError: ignored